# Network Flow Data Cleaning
This notebook cleans network flow data by standardizing direction values and removing duplicates.

## 0. Import Libraries & uploading dataset

In [10]:
import pandas as pd
import os

In [11]:
# Uploading the dataset
path = "../data"
filename = "capture20110810.csv"
fullpath = os.path.join(path, filename)

data = pd.read_csv(fullpath, sep=",")
data['sTos'] = data['sTos'].astype('Int64')
data['dTos'] = data['dTos'].astype('Int64')

print("Shape", data.shape)

Shape (2824636, 15)


## 1. Checking and fixing bad labels

In [12]:
# Check unique values in the Dir column
print("Unique direction values:")
print(data['Dir'].value_counts())

# Remove any whitespace
data['Dir'] = data['Dir'].str.strip()

# Drop row with invalid 'Dir' value: 'who'
data = data[data['Dir'] != 'who']

# Swap position of SrcAddr / DstAddr and Sport / Dport when Dir is '<-' or '<?'
mask = data['Dir'].isin(['<-', '<?'])
data.loc[mask, ['SrcAddr', 'DstAddr']] = data.loc[mask, ['DstAddr', 'SrcAddr']].values
data.loc[mask, ['Sport', 'Dport']] = data.loc[mask, ['Dport', 'Sport']].values


# Substitution of bad tokens in 'Dir' column and labeling:
    # ?>, ->, <? and <-    in  'mono'  as monodirectional flow
    # <?> and <->          in  'bi'    as bidirectional flow
data['Dir'] = data['Dir'].str.strip()   # Remove any whitespace
data['Dir'] = data['Dir'].replace({
        '->': 'mono',
        '?>': 'mono',
        '<-': 'mono',
        '<?': 'mono',
        '<->': 'bi',
        '<?>': 'bi'
    })


# Check unique values in the Dir column after cleaning
print("\nUnique direction values after cleaning:")
print(data['Dir'].value_counts())

 


Unique direction values:
Dir
<->    2191885
 ->     614958
<-        7065
<?>       6176
 ?>       4066
who        481
<?           5
Name: count, dtype: int64

Unique direction values after cleaning:
Dir
bi      2198061
mono     626094
Name: count, dtype: int64


# 2. Removing Duplicates

In [13]:
# Check number of duplicates and remove them
duplicateRows = data.duplicated().sum()
print("Numero di duplicati: ", duplicateRows)
if duplicateRows > 0:
    nodup_dff = data.drop_duplicates()
    print("New shape after removing duplicates: ", nodup_dff.shape)

Numero di duplicati:  0


# 3. Drop not useful columns

In [14]:
# Check unique values in the Dir column
print("Unique sTos and dTos values:")
sTos_values = data['sTos'].value_counts(dropna=False)
dTos_values = data['dTos'].value_counts(dropna=False)

sTos_valid_non_zero_mask = (sTos_values.index != 0) & (sTos_values.index.notna())
sTos_non_zero = sTos_values[sTos_valid_non_zero_mask].sum()
sTos_perc = sTos_non_zero/data.shape[0] * 100

dTos_valid_non_zero_mask = (dTos_values.index != 0) & (dTos_values.index.notna())
dTos_non_zero = dTos_values[dTos_valid_non_zero_mask].sum()
dTos_perc = dTos_non_zero/data.shape[0] * 100

print(f"Non-zero, non-NaN sTos occurrences: {sTos_non_zero}, \tfor {sTos_perc:.2f}% of the total dataset.")
print(f"Non-zero, non-NaN dTos occurrences: {dTos_non_zero}, \tfor {dTos_perc:.2f}% of the total dataset.")



Unique sTos and dTos values:
Non-zero, non-NaN sTos occurrences: 3323, 	for 0.12% of the total dataset.
Non-zero, non-NaN dTos occurrences: 498, 	for 0.02% of the total dataset.


In [15]:
# Drop sTos and dTos since are a minimum part of the dataset
cols_to_drop = ['sTos', 'dTos']
data.drop(columns=[c for c in cols_to_drop if c in data.columns], inplace=True, axis=1)
print(data.shape)

(2824155, 13)


# 4. Uniform labels into *botnet* (1) and *no botnet* (0)

In [16]:
# We use a lambda function to check if "botnet" is in the string (case insensitive)
# If 'botnet' appears -> 1. Everything else (Background, Safe, Normal) -> 0
data['Label'] = data['Label'].apply(lambda x: 1 if 'botnet' in str(x).lower() else 0)
print(data['Label'].value_counts())
print(data.head())

Label
0    2783194
1      40961
Name: count, dtype: int64
                    StartTime       Dur Proto        SrcAddr Sport   Dir  \
0  2011/08/10 09:46:59.607825  1.026539   tcp  94.44.127.113  1577  mono   
1  2011/08/10 09:47:00.634364  1.009595   tcp  94.44.127.113  1577  mono   
2  2011/08/10 09:47:48.185538  3.056586   tcp   147.32.86.89  4768  mono   
3  2011/08/10 09:47:48.230897  3.111769   tcp   147.32.86.89  4788  mono   
4  2011/08/10 09:47:48.963351  3.083411   tcp   147.32.86.89  4850  mono   

        DstAddr Dport State  TotPkts  TotBytes  SrcBytes  Label  
0  147.32.84.59  6881  S_RA        4       276       156      0  
1  147.32.84.59  6881  S_RA        4       276       156      0  
2   77.75.73.33    80  SR_A        3       182       122      0  
3   77.75.73.33    80  SR_A        3       182       122      0  
4   77.75.73.33    80  SR_A        3       182       122      0  


## 5. Save cleaned dataset
Specify an output file path.

In [ ]:
# When saving file with same name, it overwrites the original one.
new_filename = "cleaned_" + filename
new_path = os.path.join(path, new_filename)

data.to_csv(new_path, index=False)
print(f"Original path: {path}")
print(f"Saved cleaned data to: {new_path}")

Original path: ../data
Saved cleaned data to: ../data/cleaned_capture20110810.csv


In [18]:
# Display the first few rows of the cleaned data
data

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/10 09:46:59.607825,1.026539,tcp,94.44.127.113,1577,mono,147.32.84.59,6881,S_RA,4,276,156,0
1,2011/08/10 09:47:00.634364,1.009595,tcp,94.44.127.113,1577,mono,147.32.84.59,6881,S_RA,4,276,156,0
2,2011/08/10 09:47:48.185538,3.056586,tcp,147.32.86.89,4768,mono,77.75.73.33,80,SR_A,3,182,122,0
3,2011/08/10 09:47:48.230897,3.111769,tcp,147.32.86.89,4788,mono,77.75.73.33,80,SR_A,3,182,122,0
4,2011/08/10 09:47:48.963351,3.083411,tcp,147.32.86.89,4850,mono,77.75.73.33,80,SR_A,3,182,122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824631,2011/08/10 15:54:07.352393,0.000393,udp,147.32.86.92,36363,bi,147.32.80.9,53,CON,2,208,79,0
2824632,2011/08/10 15:54:07.353854,0.000935,udp,58.165.41.84,60122,bi,147.32.84.229,13363,CON,2,539,75,0
2824633,2011/08/10 15:54:07.357302,0.000000,tcp,147.32.84.171,47077,mono,78.191.168.43,13754,S_,1,74,74,0
2824634,2011/08/10 15:54:07.366830,0.002618,udp,93.79.39.15,10520,bi,147.32.84.229,13363,CON,2,520,460,0
